In [6]:
import cv2
import numpy as np
import os
from google.colab import drive
import shutil

In [7]:
# Montar Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
# Mapeo entre carpetas reales y nombres de estados
folder_to_state = {
    "Florecimiento": "Florecimiento Pleno",
    "Cambio": "Cambio de flores",
    "Palida": "Falta de Luz"
}

# Ruta base en Google Drive
base_path = "/content/drive/MyDrive/Fotos"  # Ajusta si es necesario

In [9]:
print(f"Buscando carpetas en: {base_path}")
print("Se procesarán las siguientes carpetas:")
for folder, state in folder_to_state.items():
    print(f"- {folder} → {state}")

Buscando carpetas en: /content/drive/MyDrive/Fotos
Se procesarán las siguientes carpetas:
- Florecimiento → Florecimiento Pleno
- Cambio → Cambio de flores
- Palida → Falta de Luz


In [10]:
# Verificar que existe el directorio base
if not os.path.exists(base_path):
    print(f"¡ERROR! No se encontró la carpeta base: {base_path}")
    print("Por favor, verifica la ruta correcta a la carpeta 'Fotos' en tu Google Drive.")
else:
    # Crear archivo dataset
    output_file = "dataset_plantas.txt"
    with open(output_file, "w") as f:
        # Encabezado
        f.write("nombre_imagen,estado,histograma_tonos\n")

        # Procesar cada carpeta
        for folder_name, state_name in folder_to_state.items():
            folder_path = os.path.join(base_path, folder_name)

            # Verificar si la carpeta existe
            if not os.path.exists(folder_path):
                print(f"¡Advertencia! No se encontró la carpeta: {folder_name}")
                continue

            print(f"\nProcesando imágenes de la carpeta '{folder_name}' como estado '{state_name}'")

            # Contador para seguimiento
            processed = 0
            errors = 0

            # Listar todas las imágenes en esta carpeta
            image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
            total_images = len(image_files)
            print(f"Encontradas {total_images} imágenes para procesar")

            for filename in image_files:
                try:
                    # Ruta completa
                    image_path = os.path.join(folder_path, filename)

                    # Cargar y procesar imagen
                    image = cv2.imread(image_path)
                    if image is None:
                        print(f"No se pudo cargar: {filename}")
                        errors += 1
                        continue

                    # Convertir a RGB y HSV
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                    image_hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)

                    # Histograma de tonos
                    hue_channel = image_hsv[:, :, 0]
                    hist_hue = cv2.calcHist([hue_channel], [0], None, [180], [0, 180])

                    # Convertir a string
                    hist_str = " ".join([str(int(val[0])) for val in hist_hue])

                    # Guardar en dataset
                    f.write(f"{filename},{state_name},{hist_str}\n")

                    processed += 1
                    if processed % 50 == 0:
                        print(f"Progreso: {processed}/{total_images} imágenes ({processed/total_images*100:.1f}%)")

                except Exception as e:
                    print(f"Error con {filename}: {str(e)}")
                    errors += 1

            print(f"Completado estado '{state_name}': {processed} imágenes procesadas, {errors} con errores")



Procesando imágenes de la carpeta 'Florecimiento' como estado 'Florecimiento Pleno'
Encontradas 300 imágenes para procesar
Progreso: 50/300 imágenes (16.7%)
Progreso: 100/300 imágenes (33.3%)
Progreso: 150/300 imágenes (50.0%)
Progreso: 200/300 imágenes (66.7%)
Progreso: 250/300 imágenes (83.3%)
Progreso: 300/300 imágenes (100.0%)
Completado estado 'Florecimiento Pleno': 300 imágenes procesadas, 0 con errores

Procesando imágenes de la carpeta 'Cambio' como estado 'Cambio de flores'
Encontradas 300 imágenes para procesar
Progreso: 50/300 imágenes (16.7%)
Progreso: 100/300 imágenes (33.3%)
Progreso: 150/300 imágenes (50.0%)
Progreso: 200/300 imágenes (66.7%)
Progreso: 250/300 imágenes (83.3%)
Progreso: 300/300 imágenes (100.0%)
Completado estado 'Cambio de flores': 300 imágenes procesadas, 0 con errores

Procesando imágenes de la carpeta 'Palida' como estado 'Falta de Luz'
Encontradas 300 imágenes para procesar
Progreso: 50/300 imágenes (16.7%)
Progreso: 100/300 imágenes (33.3%)
Progre

In [11]:
    # Descargar dataset
    from google.colab import files
    files.download(output_file)
    print(f"\nDataset completo guardado y descargado como '{output_file}'")
    print("El dataset contiene el nombre de la imagen, el estado de la planta y los valores del histograma de tonos.")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Dataset completo guardado y descargado como 'dataset_plantas.txt'
El dataset contiene el nombre de la imagen, el estado de la planta y los valores del histograma de tonos.
